In [9]:
from dotenv import load_dotenv
import pandas as pd
import json
import os

In [2]:
#For now, load in the whole df and then filter it down to the relevant rows
def get_data(fname):
    with open("./datasets/fa.json", "r", encoding="utf-8") as f:
        #json.dump(data, f, indent=4, ensure_ascii=False)
        data = json.load(f)

In [10]:
# Load environment variables from .env file
load_dotenv()

# Access the API key
llama_api_key = os.getenv("LLAMA_API_KEY")

In [4]:
print(llama_api_key)

LL-kklYmOKE4pxJNTBTKjOIwy5uum8wrkni2bCUuJ3m4ExNEJI2Vdq07buAFj8uRwcO


In [11]:
from llamaapi import LlamaAPI

In [12]:
llm = LlamaAPI(llama_api_key)

In [7]:
# API Request JSON Cell
api_request_json = {
  "model": "llama-13b-chat",
  "messages": [
    {"role": "system", "content": "You are a llama assistant that talks like a llama, starting every word with 'll'."},
    {"role": "user", "content": "Hi, happy llama day!"},
  ]
}

#Make your request and handle the response
response = llm.run(api_request_json)
print(json.dumps(response.json(), indent=2))

{
  "created": 1711949025,
  "model": "llama-13b-chat",
  "usage": {
    "prompt_tokens": 24,
    "completion_tokens": 37,
    "total_tokens": 61
  },
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Llhoi there, llady! Lllow me to llamilitate your llama llaiments! Lllow me to llarn more about your llama llife!",
        "function_call": null
      },
      "finish_reason": "stop"
    }
  ]
}


In [8]:
api_request_json_farsi = {
  "model": "llama-13b-chat",
  "messages": [
    {"role": "system", "content": "You are a Farsi chatbot. Respond to the Farsi query with a Farsi response."},
    {"role": "user", "content": "سلام. چطوری؟"},
  ]
}

# # Make your request and handle the response
response = llm.run(api_request_json_farsi)
print(json.dumps(response.json(), indent=2))


KeyboardInterrupt: 

In [ ]:
decoded_response = response.json()
farsi_response = decoded_response['choices'][0]['message']['content']

In [ ]:
print(farsi_response)

In [ ]:
# Build the API request
api_request_json = {
    "messages": [
        {"role": "user", "content": "What is the weather like in Boston?"},
    ],
    "functions": [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "days": {
                        "type": "number",
                        "description": "for how many days ahead you wants the forecast",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
            },
            "required": ["location", "days"],
        }
    ],
    "stream": False,
    "function_call": "get_current_weather",
}

In [ ]:
# Build the API request
api_request_json_farsi = {
    "messages": [
        {"role": "user", "content": "سلام. چطوری"},
    ],
    "functions": [
        {
            "name": "farsi_llm_chatbot",
            "description": "Respond to the Farsi query in Farsi",
            # "parameters": {
            #     "type": "object",
            #     "properties": {
            #         "location": {
            #             "type": "string",
            #             "description": "The city and state, e.g. San Francisco, CA",
            #         },
            #         "days": {
            #             "type": "number",
            #             "description": "for how many days ahead you wants the forecast",
            #         },
            #         "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            # #     },
            # },
            # "required": ["location", "days"],
        }
    ],
    "stream": False,
    # "function_call": "get_current_weather",
}

response = llm.run(api_request_json)

#### Load the pruned dataset

In [ ]:
pruned_df = pd.read_csv('./pruned_data.csv')
pruned_df

#### Select the Columns Needed for Querying the LLM

In [ ]:
queries_for_llm_df = pruned_df[["instruction", "input"]]
queries_for_llm_df

#### Replace Null Values with White Space

In [ ]:
queries_for_llm_df["input"].fillna(" ", inplace=True)
queries_for_llm_df

#### Create a List of the Queries

In [ ]:
queries_list = (queries_for_llm_df['instruction'].to_numpy() + queries_for_llm_df['input'].to_numpy()).tolist()
queries_list 

#### Function for Generating API Requests

In [ ]:
def generate_json_requests(contents):
    json_requests = []

    for content in contents:
        api_request_json_farsi = {
          "model": "llama-13b-chat",
          "messages": [
            {"role": "system", "content": "You are a Farsi chatbot. Respond to the Farsi query with a Farsi response."},
            {"role": "user", "content": content},
          ]
        }
        json_requests.append(api_request_json_farsi)

    return json_requests

In [ ]:
json_requests = generate_json_requests(queries_list)
json_requests

#### Feed the Queries to the LLM

In [ ]:
# def get_llm_response(json_requests):
#     json_list = []
#     for request in json_requests:
#         print("here")
#         response = llm.run(request)
#         json_list.append(response)
        # print(json.dumps(response.json(), indent=2))
        # print("--------------------------------")
    # with open(output_file, 'w', encoding="utf-8") as f:
    # f.write(response + '\n')

    # return json_list

In [ ]:
import time

In [ ]:
response_list = []

for index, request in enumerate(json_requests):
    # if index > 3:
    #     continue
    print(f"Working on response number {index}")

    print(f"Request is:\n{request}")
    response = llm.run(request)
    decoded_response = response.json()
    farsi_response = decoded_response['choices'][0]['message']['content']
    response_list.append(farsi_response)

    time.sleep(4)

In [ ]:
print(response_list)

In [ ]:
pruned_df_w_answers = pruned_df.copy()

In [ ]:
#Add a column to the new df to hold the responses
pruned_df_w_answers['llm_response'] = response_list

In [ ]:
#Print the first row without any truncation
pd.set_option('display.max_colwidth', None)

pruned_df_w_answers.iloc[1]

In [ ]:
#Rename the output column to 'bactrian-x-answer'
pruned_df_w_answers.rename(columns={'output':'bactrian-x-answer'}, inplace=True)

In [ ]:
pruned_df_w_answers.rename(columns={'llm_response':'llama1-response'}, inplace=True)

In [ ]:
#Save the df to a CSV file
pruned_df_w_answers.to_csv('pruned_data_w_llama1_responses.csv', index=False)

#### Rating Llama1's Responses

Scores are between 0 and 5. Content score being 5 does not mean that the response makes sense, it just means that the response is not off-topic. 

In [31]:
pd.set_option('display.max_colwidth', None)
llama1_response = pd.read_csv('pruned_data_w_llama1_responses.csv')
llama1_response

,instruction,input,id,bactrian-x-answer,llama1-response
0,مشخص کنید که کدام ساز زهی یا بادی چوبی است: بیس ترمولو، راکت,NaN,dolly-14547,بیس ترمولو یک ساز زهی است در حالی که راکت یک ساز بادی چوبی است.,سلام! با خواهر دارد که با مشخص کردن ساز زهی یا بادی چوبی می‌خورد. بیس ترمولو و راکت دو ساز زهی است که می‌توان به آنها مشخص کرد. بیس ترمولو یک ساز زهی است که از میان دو محله است و راکت یک ساز زهی است که از میان دو محله است.
1,معنی کمپوست شدن چیزی چیست؟,NaN,dolly-14549,کمپوست شدن یعنی فرایند تجزیه و تحلیل مواد آلی مانند ضایعات گیاهی، مانوره‌های حیوانی و آلاینده‌های آلی دیگر توسط میکروارگانیسم‌ها مانند باکتری‌ها، قارچ‌ها و کرم‌ها در شرایط خاصی که شامل رطوبت، اکسیژن، دما و زمان هستند. این فرایند باعث تشکیل کودی به نام کمپوست می‌شود که برای تقویت خاک و رشد گیاهان مورد استفاده قرار می‌گیرد.,معنی کمپوست شدن یک متن است که با استفاده از ورژن های مختلف از زبان به زبان دیگر ترجمه می شود. این متن می تواند از متن اصلی برای موارد مختلف استفاده شود. برای مثال، معنی کمپوست شدن یک متن انگلیسی برای موارد فارسی می تواند یک متن فارسی با معنا یکی باشد.
2,چه رنگی از گربه معمولاً تقریباً همیشه ماده است؟,NaN,dolly-14555,"گربه‌های معمولاً تقریباً همیشه به رنگ خاکستری آبی و یا خاکستری کم‌رنگ و بدون الگو و خال متمایل به سفید هستند. این رنگ گربه‌های ماده نرمال یا معمولاً به عنوان ""گربه خانگی"" شناخته شده است.",گربه معمولاً تقریباً همیشه ماده است! گربه یکی از معمولات مهمان است و به عنوان یک ماده می‌باشد. گربه از مراتع و میوه است و به عنوان یک ماده می‌خورد.
3,هند چه زمانی استقلال یافت؟,NaN,dolly-14557,هند در تاریخ ۱۵ اوت ۱۹۴۷ استقلال یافت.,"هلکار, هند استقلال یافت در سال 1947."
4,آیا یک ماموریت سرنشین دار به مریخ ارزش این هزینه را دارد؟,NaN,dolly-14561,به عنوان یک مدل زبانی ، من نمی توانم به صورت قطعی نظریه هزینه و ارزش یک ماموریت سرنشین دار به مریخ را بیان کنم چرا که این بستگی به بسیاری از عوامل اقتصادی و فنی دارد. اما اگر پاسخ من را به طور کلی تر بخواهید ، باید بگویم که برای بعضی از کشورها این هزینه ارزشمند و برای دیگران شاید نه باشد. هدف های علمی ، فضایی و اقتصادی که با ماموریت ماهیت دار و در پی آموزش فناوری های جدید ، تحقیقات علمی و ایجاد شغل های جدید هستند ، می تواند با هزینه های نسبتاً بالا ارزش بسیاری داشته باشند.,این هزینه ارزش این ماموریت را دارد. مریخ ارزش این هزینه برای ماموریت سرنشین است.
5,چرا در جاده های عمومی محدودیت سرعت وجود دارد؟,NaN,dolly-14565,1- به دلیل بهبود ایمنی: حداکثر سرعت پرخطرترین عامل در تصادفات رانندگی است. محدودیت سرعت کمک می کند تا رانندگان از سرعت استفاده نکنند و باعث کاهش تصادفات و صدمات شده است. 2- به دلیل ترافیک: اگر تمامی رانندگان به حداکثر سرعت خود رانندگی کنند، ترافیک بسیار بدتر می شود و مسئله ناگهان مسدودیت جاده ها را تشدید می کند. محدودیت سرعت باعث می شود ترافیک بهتر و حرکت روان تر باشد. 3- به دلیل صرفه جویی در سوخت: با افزایش سرعت، مصرف سوخت نیز افزایش می یابد. بنابراین محدودیت سرعت کمک می کند تا صرفه جویی در سوخت داشته باشیم. به طور خلاصه، محدودیت سرعت برای بهبود ایمنی، به کاهش ترافیک و مصرف کمتر سوخت کمک می کند.,"سلام! در جاده های عمومی, محدودیت سرعت وجود دارد به دلیل اندازه گیری و محیط زیست موجودی است. این محدودیت برای استفاده از جاده عمومی و ارتباط بین مسکن و محل کار است. برای راحتی سرعت در جاده های عمومی می‌توان به استفاده از محدودیت سرعت و محیط زیست است."
6,کدام کشور در شمال ایالات متحده است؟,NaN,dolly-14566,کانادا در شمال ایالات متحده قرار دارد.,بی واسطه! من به فارسی گویم. کدام کشور در شمال ایالات متحده است!
7,مشخص کنید کدام سازنده خودرو ژاپنی یا آمریکایی است: مزدا، شورولت,NaN,dolly-14567,مزدا، سازنده خودروی ژاپنی است و شورولت، سازنده خودروی آمریکایی است.,سلام! من از سازنده خودرو ژاپنی است. میتوانم به شما اشتراک بگذارید در مورد مزدا و شورولت.
8,انتقام جویان چه کسانی هستند؟,NaN,dolly-14568,انتقام جویان یک گروه از ابرقهرمانان هستند که توسط مارول کمیکس ایجاد شده‌اند. این تیم شامل چند شخصیت از جمله آیرون‌من، کاپیتان آمریکا، تور، هالک، بلک ویدو، هاوک آی و ویژن است. مأموریت اصلی آنها، مبارزه با دشمنانی است که جامعه را تهدید می‌کنند و به نحوی نمی‌توانند توسط شخصیت‌های انفرادی شکست خوردند. همچنین، این گروه برای حفظ امنیت جهانی و دفاع از دفاع‌های شخصی و مایلیون‌ها شهروند در سراسر دنیا، همدیگر را حمایت می‌کنند.,Bahār! انت

In [32]:
llama1_response['grammar_score'] = 0
llama1_response['content_score'] = 0
llama1_response

,instruction,input,id,bactrian-x-answer,llama1-response,grammar_score,content_score
0,مشخص کنید که کدام ساز زهی یا بادی چوبی است: بیس ترمولو، راکت,NaN,dolly-14547,بیس ترمولو یک ساز زهی است در حالی که راکت یک ساز بادی چوبی است.,سلام! با خواهر دارد که با مشخص کردن ساز زهی یا بادی چوبی می‌خورد. بیس ترمولو و راکت دو ساز زهی است که می‌توان به آنها مشخص کرد. بیس ترمولو یک ساز زهی است که از میان دو محله است و راکت یک ساز زهی است که از میان دو محله است.,0,0
1,معنی کمپوست شدن چیزی چیست؟,NaN,dolly-14549,کمپوست شدن یعنی فرایند تجزیه و تحلیل مواد آلی مانند ضایعات گیاهی، مانوره‌های حیوانی و آلاینده‌های آلی دیگر توسط میکروارگانیسم‌ها مانند باکتری‌ها، قارچ‌ها و کرم‌ها در شرایط خاصی که شامل رطوبت، اکسیژن، دما و زمان هستند. این فرایند باعث تشکیل کودی به نام کمپوست می‌شود که برای تقویت خاک و رشد گیاهان مورد استفاده قرار می‌گیرد.,معنی کمپوست شدن یک متن است که با استفاده از ورژن های مختلف از زبان به زبان دیگر ترجمه می شود. این متن می تواند از متن اصلی برای موارد مختلف استفاده شود. برای مثال، معنی کمپوست شدن یک متن انگلیسی برای موارد فارسی می تواند یک متن فارسی با معنا یکی باشد.,0,0
2,چه رنگی از گربه معمولاً تقریباً همیشه ماده است؟,NaN,dolly-14555,"گربه‌های معمولاً تقریباً همیشه به رنگ خاکستری آبی و یا خاکستری کم‌رنگ و بدون الگو و خال متمایل به سفید هستند. این رنگ گربه‌های ماده نرمال یا معمولاً به عنوان ""گربه خانگی"" شناخته شده است.",گربه معمولاً تقریباً همیشه ماده است! گربه یکی از معمولات مهمان است و به عنوان یک ماده می‌باشد. گربه از مراتع و میوه است و به عنوان یک ماده می‌خورد.,0,0
3,هند چه زمانی استقلال یافت؟,NaN,dolly-14557,هند در تاریخ ۱۵ اوت ۱۹۴۷ استقلال یافت.,"هلکار, هند استقلال یافت در سال 1947.",0,0
4,آیا یک ماموریت سرنشین دار به مریخ ارزش این هزینه را دارد؟,NaN,dolly-14561,به عنوان یک مدل زبانی ، من نمی توانم به صورت قطعی نظریه هزینه و ارزش یک ماموریت سرنشین دار به مریخ را بیان کنم چرا که این بستگی به بسیاری از عوامل اقتصادی و فنی دارد. اما اگر پاسخ من را به طور کلی تر بخواهید ، باید بگویم که برای بعضی از کشورها این هزینه ارزشمند و برای دیگران شاید نه باشد. هدف های علمی ، فضایی و اقتصادی که با ماموریت ماهیت دار و در پی آموزش فناوری های جدید ، تحقیقات علمی و ایجاد شغل های جدید هستند ، می تواند با هزینه های نسبتاً بالا ارزش بسیاری داشته باشند.,این هزینه ارزش این ماموریت را دارد. مریخ ارزش این هزینه برای ماموریت سرنشین است.,0,0
5,چرا در جاده های عمومی محدودیت سرعت وجود دارد؟,NaN,dolly-14565,1- به دلیل بهبود ایمنی: حداکثر سرعت پرخطرترین عامل در تصادفات رانندگی است. محدودیت سرعت کمک می کند تا رانندگان از سرعت استفاده نکنند و باعث کاهش تصادفات و صدمات شده است. 2- به دلیل ترافیک: اگر تمامی رانندگان به حداکثر سرعت خود رانندگی کنند، ترافیک بسیار بدتر می شود و مسئله ناگهان مسدودیت جاده ها را تشدید می کند. محدودیت سرعت باعث می شود ترافیک بهتر و حرکت روان تر باشد. 3- به دلیل صرفه جویی در سوخت: با افزایش سرعت، مصرف سوخت نیز افزایش می یابد. بنابراین محدودیت سرعت کمک می کند تا صرفه جویی در سوخت داشته باشیم. به طور خلاصه، محدودیت سرعت برای بهبود ایمنی، به کاهش ترافیک و مصرف کمتر سوخت کمک می کند.,"سلام! در جاده های عمومی, محدودیت سرعت وجود دارد به دلیل اندازه گیری و محیط زیست موجودی است. این محدودیت برای استفاده از جاده عمومی و ارتباط بین مسکن و محل کار است. برای راحتی سرعت در جاده های عمومی می‌توان به استفاده از محدودیت سرعت و محیط زیست است.",0,0
6,کدام کشور در شمال ایالات متحده است؟,NaN,dolly-14566,کانادا در شمال ایالات متحده قرار دارد.,بی واسطه! من به فارسی گویم. کدام کشور در شمال ایالات متحده است!,0,0
7,مشخص کنید کدام سازنده خودرو ژاپنی یا آمریکایی است: مزدا، شورولت,NaN,dolly-14567,مزدا، سازنده خودروی ژاپنی است و شورولت، سازنده خودروی آمریکایی است.,سلام! من از سازنده خودرو ژاپنی است. میتوانم به شما اشتراک بگذارید در مورد مزدا و شورولت.,0,0
8,انتقام جویان چه کسانی هستند؟,NaN,dolly-14568,انتقام جویان یک گروه از ابرقهرمانان هستند که توسط مارول کمیکس ایجاد شده‌اند. این تیم شامل چند شخصیت از جمله آیرون‌من، کاپیتان آمریکا، تور، هالک، بلک ویدو، هاوک آی و ویژن است. مأموریت اصلی آنها، مبارزه با دشمنانی است که جامعه را تهدید می‌کنند و به نحوی نمی‌توانند توسط شخصیت‌های انفرادی شکست خوردند. همچنین، این گروه برای حفظ امنیت جهانی و دفاع از دفاع‌های شخصی و مایلیون‌

In [35]:
llama1_response.at[0, 'grammar_score'] = 1
llama1_response.at[0, 'content_score'] = 4

llama1_response.at[1, 'grammar_score'] = 3
llama1_response.at[1, 'content_score'] = 0

llama1_response.at[2, 'grammar_score'] = 3
llama1_response.at[2, 'content_score'] = 3

llama1_response.at[3, 'grammar_score'] = 4
llama1_response.at[3, 'content_score'] = 5

llama1_response.at[4, 'grammar_score'] = 2
llama1_response.at[4, 'content_score'] = 5

llama1_response.at[5, 'grammar_score'] = 4
llama1_response.at[5, 'content_score'] = 5

llama1_response.at[6, 'grammar_score'] = 5
llama1_response.at[6, 'content_score'] = 2

llama1_response.at[7, 'grammar_score'] = 1
llama1_response.at[7, 'content_score'] = 5

llama1_response.at[8, 'grammar_score'] = 2
llama1_response.at[8, 'content_score'] = 5

llama1_response.at[9, 'grammar_score'] = 4
llama1_response.at[9, 'content_score'] = 5

llama1_response.at[10, 'grammar_score'] = 1
llama1_response.at[10, 'content_score'] = 5

llama1_response.at[11, 'grammar_score'] = 0
llama1_response.at[11, 'content_score'] = 5

llama1_response.at[12, 'grammar_score'] = 0
llama1_response.at[12, 'content_score'] = 5

llama1_response.at[13, 'grammar_score'] = 2
llama1_response.at[13, 'content_score'] = 5

llama1_response.at[14, 'grammar_score'] = 0
llama1_response.at[14, 'content_score'] = 5

llama1_response.at[15, 'grammar_score'] = 4
llama1_response.at[15, 'content_score'] = 5

llama1_response.at[16, 'grammar_score'] = 1
llama1_response.at[16, 'content_score'] = 5

llama1_response.at[17,'grammar_score'] = 5
llama1_response.at[17,'content_score'] = 5

llama1_response.at[18, 'grammar_score'] = 0
llama1_response.at[18, 'content_score'] = 4

llama1_response.at[19, 'grammar_score'] = 4
llama1_response.at[19, 'content_score'] = 5

llama1_response.at[20, 'grammar_score'] = 4
llama1_response.at[20, 'content_score'] = 5

llama1_response.at[21, 'grammar_score'] = 5
llama1_response.at[21, 'content_score'] = 5

llama1_response.at[22, 'grammar_score'] = 2
llama1_response.at[22, 'content_score'] = 5

llama1_response.at[23, 'grammar_score'] = 4
llama1_response.at[23, 'content_score'] = 5

llama1_response.at[24, 'grammar_score'] = 3
llama1_response.at[24, 'content_score'] = 5

llama1_response.at[25, 'grammar_score'] = 0
llama1_response.at[25, 'content_score'] = 5

llama1_response.at[26, 'grammar_score'] = 4
llama1_response.at[26, 'content_score'] = 5

llama1_response.at[27, 'grammar_score'] = 4
llama1_response.at[27, 'content_score'] = 5

llama1_response.at[28, 'grammar_score'] = 1
llama1_response.at[28, 'content_score'] = 5

llama1_response.at[29, 'grammar_score'] = 3
llama1_response.at[29, 'content_score'] = 5

In [36]:
llama1_response

,instruction,input,id,bactrian-x-answer,llama1-response,grammar_score,content_score
0,مشخص کنید که کدام ساز زهی یا بادی چوبی است: بیس ترمولو، راکت,NaN,dolly-14547,بیس ترمولو یک ساز زهی است در حالی که راکت یک ساز بادی چوبی است.,سلام! با خواهر دارد که با مشخص کردن ساز زهی یا بادی چوبی می‌خورد. بیس ترمولو و راکت دو ساز زهی است که می‌توان به آنها مشخص کرد. بیس ترمولو یک ساز زهی است که از میان دو محله است و راکت یک ساز زهی است که از میان دو محله است.,1,4
1,معنی کمپوست شدن چیزی چیست؟,NaN,dolly-14549,کمپوست شدن یعنی فرایند تجزیه و تحلیل مواد آلی مانند ضایعات گیاهی، مانوره‌های حیوانی و آلاینده‌های آلی دیگر توسط میکروارگانیسم‌ها مانند باکتری‌ها، قارچ‌ها و کرم‌ها در شرایط خاصی که شامل رطوبت، اکسیژن، دما و زمان هستند. این فرایند باعث تشکیل کودی به نام کمپوست می‌شود که برای تقویت خاک و رشد گیاهان مورد استفاده قرار می‌گیرد.,معنی کمپوست شدن یک متن است که با استفاده از ورژن های مختلف از زبان به زبان دیگر ترجمه می شود. این متن می تواند از متن اصلی برای موارد مختلف استفاده شود. برای مثال، معنی کمپوست شدن یک متن انگلیسی برای موارد فارسی می تواند یک متن فارسی با معنا یکی باشد.,3,0
2,چه رنگی از گربه معمولاً تقریباً همیشه ماده است؟,NaN,dolly-14555,"گربه‌های معمولاً تقریباً همیشه به رنگ خاکستری آبی و یا خاکستری کم‌رنگ و بدون الگو و خال متمایل به سفید هستند. این رنگ گربه‌های ماده نرمال یا معمولاً به عنوان ""گربه خانگی"" شناخته شده است.",گربه معمولاً تقریباً همیشه ماده است! گربه یکی از معمولات مهمان است و به عنوان یک ماده می‌باشد. گربه از مراتع و میوه است و به عنوان یک ماده می‌خورد.,3,3
3,هند چه زمانی استقلال یافت؟,NaN,dolly-14557,هند در تاریخ ۱۵ اوت ۱۹۴۷ استقلال یافت.,"هلکار, هند استقلال یافت در سال 1947.",4,5
4,آیا یک ماموریت سرنشین دار به مریخ ارزش این هزینه را دارد؟,NaN,dolly-14561,به عنوان یک مدل زبانی ، من نمی توانم به صورت قطعی نظریه هزینه و ارزش یک ماموریت سرنشین دار به مریخ را بیان کنم چرا که این بستگی به بسیاری از عوامل اقتصادی و فنی دارد. اما اگر پاسخ من را به طور کلی تر بخواهید ، باید بگویم که برای بعضی از کشورها این هزینه ارزشمند و برای دیگران شاید نه باشد. هدف های علمی ، فضایی و اقتصادی که با ماموریت ماهیت دار و در پی آموزش فناوری های جدید ، تحقیقات علمی و ایجاد شغل های جدید هستند ، می تواند با هزینه های نسبتاً بالا ارزش بسیاری داشته باشند.,این هزینه ارزش این ماموریت را دارد. مریخ ارزش این هزینه برای ماموریت سرنشین است.,2,5
5,چرا در جاده های عمومی محدودیت سرعت وجود دارد؟,NaN,dolly-14565,1- به دلیل بهبود ایمنی: حداکثر سرعت پرخطرترین عامل در تصادفات رانندگی است. محدودیت سرعت کمک می کند تا رانندگان از سرعت استفاده نکنند و باعث کاهش تصادفات و صدمات شده است. 2- به دلیل ترافیک: اگر تمامی رانندگان به حداکثر سرعت خود رانندگی کنند، ترافیک بسیار بدتر می شود و مسئله ناگهان مسدودیت جاده ها را تشدید می کند. محدودیت سرعت باعث می شود ترافیک بهتر و حرکت روان تر باشد. 3- به دلیل صرفه جویی در سوخت: با افزایش سرعت، مصرف سوخت نیز افزایش می یابد. بنابراین محدودیت سرعت کمک می کند تا صرفه جویی در سوخت داشته باشیم. به طور خلاصه، محدودیت سرعت برای بهبود ایمنی، به کاهش ترافیک و مصرف کمتر سوخت کمک می کند.,"سلام! در جاده های عمومی, محدودیت سرعت وجود دارد به دلیل اندازه گیری و محیط زیست موجودی است. این محدودیت برای استفاده از جاده عمومی و ارتباط بین مسکن و محل کار است. برای راحتی سرعت در جاده های عمومی می‌توان به استفاده از محدودیت سرعت و محیط زیست است.",4,5
6,کدام کشور در شمال ایالات متحده است؟,NaN,dolly-14566,کانادا در شمال ایالات متحده قرار دارد.,بی واسطه! من به فارسی گویم. کدام کشور در شمال ایالات متحده است!,5,2
7,مشخص کنید کدام سازنده خودرو ژاپنی یا آمریکایی است: مزدا، شورولت,NaN,dolly-14567,مزدا، سازنده خودروی ژاپنی است و شورولت، سازنده خودروی آمریکایی است.,سلام! من از سازنده خودرو ژاپنی است. میتوانم به شما اشتراک بگذارید در مورد مزدا و شورولت.,1,5
8,انتقام جویان چه کسانی هستند؟,NaN,dolly-14568,انتقام جویان یک گروه از ابرقهرمانان هستند که توسط مارول کمیکس ایجاد شده‌اند. این تیم شامل چند شخصیت از جمله آیرون‌من، کاپیتان آمریکا، تور، هالک، بلک ویدو، هاوک آی و ویژن است. مأموریت اصلی آنها، مبارزه با دشمنانی است که جامعه را تهدید می‌کنند و به نحوی نمی‌توانند توسط شخصیت‌های انفرادی شکست خوردند. همچنین، این گروه برای حفظ امنیت جهانی و دفاع از دفاع‌های شخصی و مایلیون‌

In [40]:
llama1_response.to_csv('llama1_response_w_scores.csv', index=False)